In [0]:
from pyspark.sql import functions as F

# =========================
# 설정
# =========================
BASE_PATH = "/Volumes/signalcraft_databricks/signalcraft_test/bronze"
CATALOG = "signalcraft_databricks"
SCHEMA  = "default"

NETFLIX_FILE = "netflix.csv"
USER_FILE    = "user.csv"

NETFLIX_TABLE = f"{CATALOG}.{SCHEMA}.bronze_netflix"
USER_TABLE    = f"{CATALOG}.{SCHEMA}.bronze_user"

# =========================
# 1) netflix.csv -> bronze_netflix
# =========================
netflix_path = f"{BASE_PATH}/{NETFLIX_FILE}"

df_netflix = (
    spark.read.format("csv")
      .option("header", "true")
      .option("inferSchema", "true")
      .load(netflix_path)
      .withColumn("_ingest_ts", F.current_timestamp())
      .withColumn("_source_file", F.lit(netflix_path))
)

(df_netflix.write
 .format("delta")
 .mode("overwrite")     # 최초 적재: overwrite / 이후 누적이면 append 고려
 .saveAsTable(NETFLIX_TABLE)
)

print(f"✅ Saved: {NETFLIX_TABLE}  (rows={df_netflix.count()})")


# =========================
# 2) user.csv -> bronze_user
# =========================
user_path = f"{BASE_PATH}/{USER_FILE}"

df_user = (
    spark.read.format("csv")
      .option("header", "true")
      .option("inferSchema", "true")
      .load(user_path)
      .withColumn("_ingest_ts", F.current_timestamp())
      .withColumn("_source_file", F.lit(user_path))
)

(df_user.write
 .format("delta")
 .mode("overwrite")
 .saveAsTable(USER_TABLE)
)

print(f"✅ Saved: {USER_TABLE}  (rows={df_user.count()})")


✅ Saved: signalcraft_databricks.default.bronze_netflix  (rows=8809)
✅ Saved: signalcraft_databricks.default.bronze_user  (rows=10000)
